## Classification avec extraction de features par CNN
programme d'extraction de caractéristiques: fait (en utilisant dinov2 modèle de META)
réduction de dimension: fait (en utilisant l'analyse en composantes principales (ACP) ) puis classification non supervisée par K-means



In [ ]:
import os
import shutil
import numpy as np
import torch
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


#Charger DINOv2
device = "cuda" if torch.cuda.is_available() else "cpu"

model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
model.to(device)
model.eval()


#Preprocessing images (les images du dataset ne sont pas toutes de la même taille pour éviter des détections de contour des bordures)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def preprocess(img_path):
    img = Image.open(img_path).convert("RGB")
    img = transform(img)
    img = img.unsqueeze(0).to(device)
    return img

#Charger les chemins des images
dataset_path = "dataset_edge"

image_paths = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))

print("Nombre d'images trouvées :", len(image_paths))


#Extraction des features DINOv2

features = []

for img_path in tqdm(image_paths):
    img = preprocess(img_path)
    with torch.no_grad():
        feat = model(img)
    features.append(feat.cpu().numpy().flatten())

X = np.array(features)
print("Shape des features :", X.shape)  # (nb_images, 384)


#PCA (réduction dimension)
pca = PCA(n_components=50)
X_reduced = pca.fit_transform(X)

print("Shape après PCA :", X_reduced.shape)


# K-MEANS

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_reduced)

labels = kmeans.labels_

print("Clustering terminé.")

#Créer les dossiers de sortie et copier les images dans les dossiers correspondants aux clusters

output_dir = "resultats_kmeans"
os.makedirs(output_dir, exist_ok=True)

for cluster_id in range(n_clusters):
    os.makedirs(os.path.join(output_dir, f"cluster_{cluster_id}"), exist_ok=True)

for img_path, label in zip(image_paths, labels):
    dest_folder = os.path.join(output_dir, f"cluster_{label}")
    shutil.copy2(img_path, os.path.join(dest_folder, os.path.basename(img_path)))

print("Images classées dans les dossiers par cluster !")

Using cache found in /Users/hamzachoukaili/.cache/torch/hub/facebookresearch_dinov2_main


Nombre d'images trouvées : 683


100%|██████████| 683/683 [00:21<00:00, 32.35it/s]


Shape des features : (683, 384)
Shape après PCA : (683, 50)
Clustering terminé.
✅ Images classées dans les dossiers par cluster !
